In [9]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import random
import os
import datetime

### 建立航運股的股票、日期

In [10]:
stock ={"陽明":"2609","長榮":"2603","萬海":"2615","長榮航":"2618","遠雄港":"5607","華航":"2610","台驊投控":"2636","裕民":"2606",
        "益航":"2601" ,"東森":"2614","新興":"2605" , "四維航":"5608" , "台航":"2617" }

In [11]:
day_now =datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m%d') #現在的月份
#建立url需要的日期部分，格式為年、月、1號 >>> 20190101
date=pd.date_range(start="20170101",end=day_now ,freq="MS" ).map(lambda x : x.strftime("%Y%m%d"))
#建立dataFRAM的index
date_index = pd.date_range(start="20170101",end=day_now ,freq="MS" ).map(lambda x : x.strftime("%Y%m"))

### 取每月的最高價、最低價、收盤中位數、成交量中位數

In [12]:
#header
headers ={"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [20]:
def get_text(url   ,headers ):
    """負責request的請求
    傳入網址、header(避免反爬蟲機制)"""
    r = requests.get(url, headers=headers )
    res = r.text
    soup=BeautifulSoup( res ,"lxml")
    return soup

In [21]:
def get_value(soup):
    """請傳入 rqquest.get(url).text"""
    
    """return 資料為一個月的數值的四個數值 max_price、min_price、median_price、median的交易筆數 """
    #使用BeautifulSoup 做解析
    
    max_p = []#每日最大的價格的集合（月）
    min_p=[]#每日最低的價格的集合（月）
    median_p=[]#每日的收盤價格的集合（月）
    median_trans_all =[]#每日的交易筆數的集合（月）
      
    for  tr in soup.find_all("tr")[2:]:
        if tr.find_all("td")[4].text == "--": #當其中值是等於該“"--"符號，跳過本次迴圈
            continue
    
        max_=float(tr.find_all("td")[4].text)#原欄位是當天最高價
        min_=float(tr.find_all("td")[5].text)#原欄位是當天最低價
        median_=float(tr.find_all("td")[6].text)#原欄位是當天收盤價
        median_tran=float(tr.find_all("td")[8].text.replace(",",""))#去除逗號、原欄位是當天的交易筆數
        max_p.append(max_)
        min_p.append(min_)
        median_p.append(median_)
        median_trans_all.append(median_tran)

    max_end =np.max(max_p)#將該當月最高價的list，找出當月的最高價
    min_end =np.min(min_p)#將該當月最低價的list，找出當月的最低價
    median_price_end=np.median(median_p)#將該當月的收盤價list，找出當月的中位數收盤價
    median_tran_end =np.median(median_trans_all)#將該當月最低價的交易筆數list，找出當月的中位數交易筆數
    
    return  max_end ,  min_end ,  median_price_end ,  median_tran_end

### 將各航運股公司的資料解析，取出所需的資料，輸出成CSV

In [ ]:
# 決定要儲存的資料夾
output_dir = 'Data_price' 

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
for name ,code  in zip(stock,stock.values()):
    url =  [f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date={j}&stockNo={code}" for  j in date ]
    #建立該公司的url
    max_=[]#所有當月最高價格
    min_=[]#所有當月最低價格
    median_p=[]#所有當月中位數的收盤價
    median_tran=[]#所有當月中位數的交易筆數
    n=0 #額外停止時間
    for url_ in url:
        n+=1
        #print(url_)
        soup = get_text(url_ ,headers )
        time.sleep(3)
        while soup is None:
            soup = get_text(url_ ,headers )
            time.sleep(3)  
        
        max_end ,  min_end ,  median_price_end ,  median_tran_end=get_value(soup)
        max_.append(max_end)#將該公司所有的當月最高價格傳入
        min_.append(min_end)#將該公司所有的當月最低價格傳入
        median_p.append(median_price_end)#將該公司所有的當月中位數的收盤價格
        median_tran.append(median_tran_end)#將該公司所有的當月中位數的交易筆數
        if n%15==0:#每15次停5秒
            time.sleep(5)
    """將每一個公司建一個DataFrame，並建立該公司資料的csv檔，索引用日期的部分"""
    pd.DataFrame({"max_price":max_,"min_price":min_,"medain_price":median_p,
                      "medain_transactions":median_tran},index= date_index).round(2).to_csv(f"./Data_price/{name}_{code}.csv")
    
    
        
        
        
    
    

##### 單一公司

In [22]:
company_Name="台航"
company_Code="2617"
url =  [f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date={j}&stockNo={company_Code}" for  j in date ]
#建立該公司的url
max_=[]#所有當月最高價格
min_=[]#所有當月最低價格
median_p=[]#所有當月中位數的收盤價
median_tran=[]#所有當月中位數的交易筆數
n=0
for url_ in url:
    n+=1
    print(url_)
    soup = get_text(url_ ,headers )
    time.sleep(3)
    while soup is None :
        soup = get_text(url_ ,headers )
        time.sleep(3)
    max_end ,  min_end ,  median_price_end ,  median_tran_end=get_value(soup)
    max_.append(max_end)#將該公司所有的當月最高價格傳入
    min_.append(min_end)#將該公司所有的當月最低價格傳入
    median_p.append(median_price_end)#將該公司所有的當月中位數的收盤價格
    median_tran.append(median_tran_end)#將該公司所有的當月中位數的交易筆數
    if n%15==0:#每15次停10秒
        time.sleep(10)
    """將每一個公司建一個DataFrame，並建立該公司資料的csv檔，索引用日期的部分"""
    
pd.DataFrame({"max_price":max_,"min_price":min_,"medain_price":median_p,
                      "medain_transactions":median_tran}
             ,index= date_index).round(2).to_csv(f"./Data_price/{company_Name}_{company_Code}.csv")
    

https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170101&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170201&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170301&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170401&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170501&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170601&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170701&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170801&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20170901&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20171001&stockNo=2617
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20171101&stockNo=2617
https://ww